In [ ]:
from utilities import *

In [ ]:
pathname = "recds_yg/"
filenames = []
for root, dirs, files in os.walk(pathname):
    for name in files:     
        filename = pathname + name
        filenames.append(filename)
            
hmm_digit = get_hmm_digits(filenames)

In [ ]:
num_filter = 40
num_states = 5
num_digits = 7

In [ ]:
## hmm_digit: 10 hmms for one digit recognized
## change transition_matrix 
for i in range(len(hmm_digit)):
    hmm_digit[i].transition_matrix[num_states-1][num_states-1] = 0.5

In [ ]:
## non emitting state
class neState:
    def __init__(self,id,transition_prob=0.5,parent=None,next=None):
        self.id = id
        self.transition_prob = transition_prob
        self.parent = parent ## prev as a list of state4
        self.next = next ## next as a list of state0

In [ ]:

## assign name to state
hmm_digits = []
for i in range(7):
    if i == 0:
        hmm_di = copy.deepcopy(hmm_digit[2:])
        for j in range(len(hmm_di)):
            hmm = hmm_di[j]
            for k in range(len(hmm.states)):
                hmm.states[k].name = num2words(j+2)
        
    else:
        hmm_di = copy.deepcopy(hmm_digit)
        for j in range(len(hmm_di)):
            hmm = hmm_di[j]
            for k in range(len(hmm.states)):
                hmm.states[k].name = num2words(j)
        
     
    hmm_digits.append(hmm_di)

In [ ]:
## nonemiting states list
nonemitingStates = []
for i in range(num_digits+1):
    nonemitingState = neState(id=i)
    #print(nonemitingState)
    nonemitingStates.append(nonemitingState)
    

In [ ]:
## reconstruct and add nonemitting state
## assign .parent and .next to state
## assign transition prob to state

for i in range(len(hmm_digits)):
    nonemitingState_prev = nonemitingStates[i]
    nonemitingState_post = nonemitingStates[i+1]
    hmm_digit= hmm_digits[i]
    
    for j in range(len(hmm_digit)):
        hmm = hmm_digit[j]
        transition_matrix = hmm.transition_matrix
        for k in range(len(hmm.states)):
            if k == 0:
                ## parent: nonemitting + self
                parent1 = (nonemitingState_prev,1)
                parent2 = (hmm.states[k],transition_matrix[k][k])   
                ## child: next + self
                children1 = (hmm.states[k+1],transition_matrix[k][k+1])
                children2 = (hmm.states[k],transition_matrix[k][k])
            elif k == num_states-1:
                ## parent: prev + self
                parent1 = (hmm.states[k-1],transition_matrix[k-1][k])
                parent2 = (hmm.states[k],transition_matrix[k][k]) 
                ## children: nonemitting + self
                children1 = (nonemitingState_post,0.5)
                children2 = (hmm.states[k],transition_matrix[k][k])
            else:
                ## parent: prev + self
                parent1 = (hmm.states[k-1],transition_matrix[k-1][k])
                parent2 = (hmm.states[k],transition_matrix[k][k]) 
                ## children: next + self
                children1 = (hmm.states[k+1],transition_matrix[k][k+1])
                children2 = (hmm.states[k],transition_matrix[k][k])
            hmm.states[k].parent = [parent1,parent2]
            hmm.states[k].next = [children1,children2]


In [ ]:
## add nonemiting state

## case when nonemitingstate0
child = []
nonemitingState0 = nonemitingStates[0]
hmm_digit0 = hmm_digits[0]  
hmm_digit3 = hmm_digits[3]
for i in range(len(hmm_digit0)):
    child.append(hmm_digit0[i].states[0])
for j in range(len(hmm_digit3)):
    child.append(hmm_digit3[j].states[0])
nonemitingState0.next = child

for i in range(1,len(nonemitingStates)):
    nonemitingState = nonemitingStates[i]
    ## connect last state of previous hmm_digit and nonemitting state
    hmm_digit_prev = hmm_digits[i-1]
    parents = []
    for j in range(len(hmm_digit_prev)):
        parents.append(hmm_digit_prev[j].states[num_states-1])
    nonemitingState.parent = parents
    ## connect nonemitting state and first state of next hmm_digit
    if i != 7:
        hmm_digit_post = hmm_digits[i]
        child = [] 
        for k in range(len(hmm_digit_post)):
            child.append(hmm_digit_post[k].states[0])
        nonemitingState.next = child
        


In [ ]:
## build tree
states = []
cnt = -2
for i in range(len(nonemitingStates)):
    cur_neState = nonemitingStates[i]
    cnt += 1
    cur_neState.id = cnt
    states.append(cur_neState)
    if i != 7:
        hmm_digit = hmm_digits[i]
        for j in range(len(hmm_digit)):
            hmm = hmm_digit[j]
            for k in range(len(hmm.states)):
                cnt += 1
                hmm.states[k].id = cnt
                states.append(hmm.states[k])

In [ ]:
nonemitingState_ids = []
for i in range(len(nonemitingStates)):
    nonemitingState_ids.append(nonemitingStates[i].id)

In [ ]:
nonemitingState_ids

In [ ]:
## states without nonemitingstate0
states = states[1:]
len(states)

In [ ]:
states[0]

In [ ]:
nonemitingStates[0].next[-1].id

In [ ]:
## children of nonemitingstate0
childNonemitingstate0 = []
for i in range(len(nonemitingStates[0].next)):
    childid = nonemitingStates[0].next[i].id
    childNonemitingstate0.append(childid)
    

In [ ]:
childNonemitingstate0

In [162]:
testfile = 'record.wav'
test = mfcc_features(testfile,num_filter)

In [163]:
type(states[0]) == State

True

In [167]:
len(test)

27

In [169]:
## dtw alignment
back_pointer_table = np.zeros((len(states),len(test)))
cost = np.zeros((len(states))) + np.inf
for j in range(len(test)):
    
    prev_cost = copy.deepcopy(cost)
    
    for i in range(len(states)):
        state = states[i]
        
        ## initialization
        ## if state is children of nonemittingState0
        ## start with hmm_digit0 and hmm_digit 3
        if j == 0:
            if state.id in childNonemitingstate0:
                cost[i] = get_node_cost(test[j],state)
                back_pointer_table[i][j] = state.id
        
        else: 
            ## if state is State
            if type(state) == State:
                
                ## if state0 of each hmm
                if (type(state.parent[0][0]) == neState):
                    
                    ## hmm_digit0 cannot inherit nonemmitingstate0
                    if state.parent[0][0].id != -1:
                    
                        ## pathcost: cost from previous nonemitting state
                        ## no edge cost due to nonemiting
                        pathcost = prev_cost[state.parent[0][0].id]

                        local_node_cost = get_node_cost(test[j],state)
                        
                        cost[i] = pathcost + local_node_cost
                    
                        back_pointer_table[i][j] = state.parent[0][0].id
                    
                    ## state0 of hmm_digit0
                    else:
                        
                     
                ## normal state   
                else:
                    ## pathcost1: from parent
                    pathcost1 = prev_cost[state.parent[0][0].id]
                    edgecost1 = -np.log(state.parent[0][1])
                    dist1 = pathcost1 + edgecost1
                    ## pathcost2: from self-transition
                    pathcost2 = prev_cost[state.parent[1][0].id]
                    edgecost2 = -np.log(state.parent[1][1])
                    dist2 = pathcost2 + edgecost2
                    
                    ## compare & assign
                    cost[i] = np.min([dist1,dist2]) + get_node_cost(test[j],state)
                    back_pointer_table[i][j] = state.parent[np.argmin([dist1,dist2])][0].id
                
                
            ## if state is nonemitting state
            else:
                ## select the min of 10 hmm in hmm_digit
                combinedcost = []
                ## cost of each 10 hmm
                for k in range(len(state.parent)):
                    costtmp = prev_cost[state.parent[k].id]
                    combinedcost.append(costtmp)
                cost[i] = np.min(combinedcost)
                back_pointer_table[i][j] = state.parent[np.argmin(combinedcost)].id
    

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   1., ...,   1.,   0.,   0.],
       [  0.,   1.,   1., ...,   2.,   2.,   1.],
       ...,
       [  0., 343., 343., ..., 343., 343., 343.],
       [  0., 344., 344., ..., 345., 344., 345.],
       [  0., 300., 300., ..., 310., 305., 335.]])

In [172]:
## trace back
state_id = len(states) - 1
state_ids = [state_id]
for j in range(len(test)-1,0,-1):
    state_id = int(back_pointer_table[state_id][j])
    state_ids.append(state_id)
state_ids = state_ids[::-1]

In [175]:
names = []
for state_id in state_ids:
    if state_id not in nonemitingState_ids:
        state = states[state_id]
        name = state.name
        names.append(name)

In [177]:
len(names)

23

In [ ]:
nonemitingStates[7].parent